Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.617807
Minibatch accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss at step 50: 1.763763
Minibatch accuracy: 50.0%
Validation accuracy: 56.5%
Minibatch loss at step 100: 0.749195
Minibatch accuracy: 75.0%
Validation accuracy: 68.1%
Minibatch loss at step 150: 0.980316
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 200: 0.234219
Minibatch accuracy: 93.8%
Validation accuracy: 75.4%
Minibatch loss at step 250: 0.538514
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 300: 0.780967
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.463289
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 400: 1.256397
Minibatch accuracy: 56.2%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.933962
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.523672
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [19]:
batch_size = 256
# print(28*28)
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size, image_size, 1))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_training = tf.placeholder(tf.bool)


    def getLogits(inputs):
        with tf.variable_scope('conv1'):
            conv1 = tf.layers.conv2d(inputs=inputs, filters=32, kernel_size=(1,1), padding='same')
            conv1 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=(1,3), padding='same')
            conv1 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=(3,1), padding='same')
            conv1 = tf.layers.batch_normalization(conv1)
            conv1 = tf.nn.relu(conv1)
            conv1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
            conv1 = tf.layers.dropout(inputs=conv1,rate=0.25,noise_shape=None,seed=None,training=tf_training)
        
        with tf.variable_scope('conv2'):
            conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=(1,1), padding='same')
            conv2 = tf.layers.conv2d(inputs=conv2, filters=64, kernel_size=(1,3), padding='same')
            conv2 = tf.layers.conv2d(inputs=conv2, filters=64, kernel_size=(3,1), padding='same')
            conv2 = tf.layers.batch_normalization(conv2)
            conv2 = tf.nn.relu(conv2)
            conv2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
            conv2 = tf.layers.dropout(inputs=conv2,rate=0.25,noise_shape=None,seed=None,training=tf_training)

        with tf.variable_scope('conv3'):
            conv3 = tf.layers.conv2d(inputs=conv2, filters=128, kernel_size=(1,1), padding='same')
            conv3 = tf.layers.conv2d(inputs=conv3, filters=128, kernel_size=(1,3), padding='same')
            conv3 = tf.layers.conv2d(inputs=conv3, filters=128, kernel_size=(3,1), padding='same')
            conv3 = tf.layers.batch_normalization(conv3)
            conv3 = tf.nn.relu(conv3)
            conv3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
            conv3 = tf.layers.dropout(inputs=conv3,rate=0.25,noise_shape=None,seed=None,training=tf_training)

            
        with tf.variable_scope('flatten'):
            flatten = tf.contrib.layers.flatten(conv3)
        
        with tf.variable_scope('dense1'):
            hidden = tf.layers.dense(inputs=flatten, units=1024)
            hidden = tf.layers.batch_normalization(hidden)
            hidden = tf.nn.relu(hidden)
            hidden = tf.layers.dropout(inputs=hidden,rate=0.25,noise_shape=None,seed=None,training=tf_training)

        with tf.variable_scope('dense2'):
            hidden = tf.layers.dense(inputs=hidden, units=256)
            hidden = tf.layers.batch_normalization(hidden)
            hidden = tf.nn.relu(hidden)
            hidden = tf.layers.dropout(inputs=hidden,rate=0.25,noise_shape=None,seed=None,training=tf_training)
        
        with tf.variable_scope('dense3'):
            hidden = tf.layers.dense(inputs=hidden, units=64)
            hidden = tf.layers.batch_normalization(hidden)
            hidden = tf.nn.relu(hidden)
            hidden = tf.layers.dropout(inputs=hidden,rate=0.25,noise_shape=None,seed=None,training=tf_training)
        
        with tf.variable_scope('dense4'):
            logits = tf.layers.dense(inputs=hidden, units=10, activation=tf.nn.relu)
            
        return logits
    
    logits = getLogits(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    tf.summary.scalar('loss', loss)
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.1
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.99, staircase=True)
    tf.summary.scalar('learning_rate', learning_rate)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(logits)

# merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('./tensorboardlog', graph)
# summary_writer.flush()

In [21]:
num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    merged = tf.summary.merge_all()
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_training: True}
        
        _, summary, predictions = session.run([optimizer, merged, train_prediction], feed_dict=feed_dict)
        summary_writer.add_summary(summary, step)
        if (step % 100 == 0):
#             print("Minibatch loss at step %d: %f" % (step, l))
            print("Learning rate: %f, Global Step: %d" % (learning_rate.eval(), global_step.eval()))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, tf_training: False}
            print("Validation accuracy: %.1f%%" % accuracy(session.run(valid_prediction, feed_dict=feed_dict), valid_labels))
    feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels, tf_training: False} 
    print("Test accuracy: %.1f%%" % accuracy(session.run(test_prediction, feed_dict=feed_dict), test_labels))
    
summary_writer.flush()

Initialized
Learning rate: 0.100000, Global Step: 1
Minibatch accuracy: 9.4%
Validation accuracy: 9.9%
Learning rate: 0.099000, Global Step: 101
Minibatch accuracy: 49.6%
Validation accuracy: 65.5%
Learning rate: 0.098010, Global Step: 201
Minibatch accuracy: 74.6%
Validation accuracy: 80.1%
Learning rate: 0.097030, Global Step: 301
Minibatch accuracy: 76.2%
Validation accuracy: 83.9%
Learning rate: 0.096060, Global Step: 401
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Learning rate: 0.095099, Global Step: 501
Minibatch accuracy: 80.9%
Validation accuracy: 85.7%
Learning rate: 0.094148, Global Step: 601
Minibatch accuracy: 83.2%
Validation accuracy: 86.0%
Learning rate: 0.093207, Global Step: 701
Minibatch accuracy: 81.6%
Validation accuracy: 86.5%
Learning rate: 0.092274, Global Step: 801
Minibatch accuracy: 85.5%
Validation accuracy: 87.1%
Learning rate: 0.091352, Global Step: 901
Minibatch accuracy: 85.5%
Validation accuracy: 86.8%
Learning rate: 0.090438, Global Step: 1001

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---